In [1]:
import pandas as pd
import numpy as np

# Load the Excel file
highway_node = pd.read_excel("HighwayNode.xlsx", header=None)

# Initialize R as a 3D numpy array
R = np.zeros((14, 24, 30))

# Function to clean and extract data from the CSV files
def process_csv(file_path):
    # Skip the first 3 rows and read the file from the 4th row
    data = pd.read_csv(file_path, skiprows=3, header=0, delimiter=',')
    
    # Convert all data to string and handle missing values
    data = data.astype(str).replace('nan', '')
    
    # Return the data as a NumPy array
    return data.values

# Iterate over each node
for k in range(1, 15):
    # Find rows where the 7th column equals k
    A = highway_node[highway_node.iloc[:, 6] == k].index.tolist()
    
    for i in A:
        # Get the file path from the 6th column of the selected row
        file_path = highway_node.iloc[i, 5]
        
        # Process the CSV file and extract the data
        Data = process_csv(file_path)
       
        # Iterate over each day and time period
        for n in range(30):
            for t in range(6, 24):
                try:
                    start_row = 96 * n + 4 * (t-1) + 4
                    end_row = start_row + 4
                    R[k-1, t-1, n] += np.sum(pd.to_numeric(Data[start_row:end_row, 4]))
                except Exception as e:
                    print(f"Error processing data for node {k}, day {n}, time {t}: {e}")
                    continue

print("R_k.t.n generated successfully.")


R_k.t.n generated successfully.


In [2]:
import numpy as np
import networkx as nx

# Define the edges (assuming node indexing starts from 1)
edges = [
    (1, 2, 1),
    (2, 3, 3.5),
    (3, 4, 14),
    (4, 5, 3.8),
    (5, 6, 17.5),
    (6, 7, 14.5),
    (7, 8, 4.1),
    (8, 9, 6.9),
    (9, 10, 15.5),
    (10, 11, 3.3),
    (11, 12, 10.7),
    (12, 13, 13),
    (13, 14, 2.7),
    (14, 1, 7.9)
]

# Create a graph
G = nx.Graph()

# Add edges to the graph
for u, v, w in edges:
    G.add_edge(u, v, weight=w)

# Ensure node labels start from 0
G = nx.convert_node_labels_to_integers(G, first_label=0)

# Initialize the distance matrix
num_nodes = G.number_of_nodes()
d = np.zeros((num_nodes, num_nodes))

# Initialize the indicator matrix
a = np.zeros((num_nodes, num_nodes, num_nodes), dtype=int)

# Compute shortest paths and distances using Dijkstra's algorithm
for i in range(num_nodes):
    # Use Dijkstra's algorithm to compute shortest paths from node i
    lengths, paths = nx.single_source_dijkstra(G, i)
    
    for j in range(num_nodes):
        if i != j:
            # Distance between node i and node j
            d[i, j] = lengths[j]
            
            # Shortest path from node i to node j
            path = paths[j]
            
            # Set indicator matrix entries
            for k in range(1, len(path) - 1):  # Exclude the source (i) and target (j) nodes
                intermediate = path[k]
                a[i, j, intermediate] = 1

In [3]:
Tij = d/100

In [4]:
from gurobipy import Model, GRB, quicksum
import math

# Parameters
N = [i for i in range(1, 15)]
C1, C2, C3 = 100, 50, 100
fc = lambda c: 200 + 50 * c
gc = lambda c: 300 + 300 * c
hc = lambda c: 200 + 30 * c
theta = 10
r = 30
TC = 2
TB = 1/6
Ttol = 1/2
epsilon = 0.8
w = 0.0365
OB, OE = 6, 23
observed_days = 30

# # Observed data (R_k.t.n) should be provided as a dictionary with keys (k,t,n)
# R = {(k,t,n): 0 for k in N for t in range(OB, OE+1) for n in range(1, observed_days+1)}

# # Distance and transportation time matrices should be provided as dictionaries
# d = {(i,j): 1 for i in N for j in N}  # Using 1 as a placeholder value
# Tij = {(i,j): 1 for i in N for j in N}  # Using 1 as a placeholder value
# a = {(i,j,k): 0 for i in N for j in N for k in N}

# Create a new model
model = Model("EV Infrastructure Planning")

# Decision variables
A = model.addVars(N, range(1, C1+1), vtype=GRB.BINARY, name="A")
B = model.addVars(N, range(1, C2+1), vtype=GRB.BINARY, name="B")
O = model.addVars(N, range(1, C3+1), vtype=GRB.BINARY, name="O")
L = model.addVars(N, vtype=GRB.BINARY, name="L")
H = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="H")
C = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="C")
M = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="M")
N_var = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="N")
F = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="F")
E = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="E")
D = model.addVars(N, N, range(OB, OE+1), vtype=GRB.INTEGER, name="D")
G = model.addVars(N, N, range(OB, OE+1), vtype=GRB.INTEGER, name="G")
I = model.addVars(N, vtype=GRB.INTEGER, name="I")
mu = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="mu")
lambda_k_t = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="lambda")

# Objective function
model.setObjective(
    quicksum(fc(c) * A[k, c] for k in N for c in range(1, C1+1)) +
    quicksum(gc(c) * B[k, c] for k in N for c in range(1, C2+1)) +
    quicksum(hc(c) * O[k, c] for k in N for c in range(1, C3+1)) +
    quicksum(w * d[i-1, j-1] * D[i, j, t] for i in N for j in N for t in range(OB, OE+1)) +
    quicksum(w * d[i-1, j-1] * G[i, j, t] for i in N for j in N for t in range(OB, OE+1)) +
    quicksum(theta * I[k] for k in N),
    GRB.MINIMIZE
)

# Constraints
model.addConstrs((quicksum(A[k, c] for c in range(1, C1+1)) <= 1 for k in N), name="C2")
model.addConstrs((quicksum(B[k, c] for c in range(1, C2+1)) <= 1 for k in N), name="C3")
model.addConstrs((quicksum(O[k, c] for c in range(1, C3+1)) <= 1 for k in N), name="C4")

model.addConstrs((L[k] <= quicksum(A[k, c] for c in range(1, C1+1)) + quicksum(B[k, c] for c in range(1, C2+1)) for k in N), name="C5")
model.addConstrs((L[k] >= 0.5 * (quicksum(A[k, c] for c in range(1, C1+1)) + quicksum(B[k, c] for c in range(1, C2+1))) for k in N), name="C6")
# a = np.zeros((14, 14, 14))
# # model.addConstrs((d[i-1, j-1] <= r * (1 + quicksum(L[k] * a[i-1, j-1, k-1] for k in N)) for i in N for j in N if i != j), name="C7")

for i in N:
    for j in N:
        if i != j:
            model.addConstr(float(d[i-1, j-1]) <= r * (1 + quicksum(L[k] * int(a[i-1, j-1, k-1]) for k in N)), name=f"C7_{i}_{j}")

model.addConstrs((F[k, OB] == I[k] for k in N), name="C8")
model.addConstrs((F[k, OE] == I[k] for k in N), name="C9")
model.addConstrs((E[k, OB] == 0 for k in N), name="C10")

model.addConstrs((
    F[k, t] == F[k, t-1] - H[k, t-1] + (M[k, t-TC] if t-TC >= OB else 0) + (N_var[k, t-TC] if t-TC >= OB else 0) -
    quicksum(D[k, j, t-1] for j in N) +
    quicksum(D[j, k, t-int(math.ceil(Tij[j-1, k-1]))] if t-int(math.ceil(Tij[j-1, k-1])) >= OB else 0 for j in N)
    for k in N for t in range(OB+1, OE+1)
), name="C11")

model.addConstrs((
    E[k, t] == E[k, t-1] + H[k, t-1] - M[k, t-1] - N_var[k, t-1] -
    quicksum(G[k, j, t-1] for j in N) +
    quicksum(G[j, k, t-int(math.ceil(Tij[j-1, k-1]))] if t-int(math.ceil(Tij[j-1, k-1])) >= OB else 0 for j in N)
    for k in N for t in range(OB+1, OE+1)
), name="C12")

model.addConstrs((M[k, t] + N_var[k, t] <= E[k, t] for k in N for t in range(OB, OE+1)), name="C13")
model.addConstrs((quicksum(M[k, l] for l in range(max(OB, t-TC+1), t+1)) <= quicksum(c * A[k, c] for c in range(1, C1+1)) for k in N for t in range(OB, OE+1)), name="C14")
model.addConstrs((quicksum(N_var[k, l] for l in range(max(OB, t-TC+1), t+1)) <= quicksum(c * O[k, c] for c in range(1, C3+1)) for k in N for t in range(OB, OE+1)), name="C15")
model.addConstrs((H[k, t] <= (1/TB) * quicksum(c * B[k, c] for c in range(1, C2+1)) for k in N for t in range(OB, OE+1)), name="C16")

model.addConstrs((quicksum(D[k, j, t] for j in N) <= F[k, t] - H[k, t] for k in N for t in range(OB, OE+1)), name="C17")
model.addConstrs((quicksum(G[k, j, t] for j in N) <= E[k, t] + H[k, t] for k in N for t in range(OB, OE+1)), name="C18")

model.addConstrs((
    quicksum(F[k, t] + E[k, t] + quicksum(M[k, l] + N_var[k, l] for l in range(max(OB, t-TC+1), t+1)) +
    quicksum(D[k, j, t-int(math.ceil(Tij[k-1, j-1]))] + G[k, j, t-int(math.ceil(Tij[k-1, j-1]))] if t-int(math.ceil(Tij[k-1, j-1])) >= OB else 0 for j in N)
    for k in N) == quicksum(I[k] for k in N)
    for t in range(OB, OE+1)
), name="C19")

model.addConstrs((C[k, t] <= (quicksum(c * A[k, c] for c in range(1, C1+1)) - quicksum(M[k, l] for l in range(max(OB, t-TC+1), t+1)) - quicksum(C[k, l] for l in range(max(OB, t-TC), t)))
                 for k in N for t in range(OB, OE+1)), name="C20")

model.addConstrs((mu[k, t] == C[k, t] + H[k, t] for k in N for t in range(OB, OE+1)), name="C21")
# model.addConstrs((quicksum(R[k-1, t-1, n-1] for n in range(1, observed_days+1)) / observed_days >= lambda_k_t[k, t] for k in N for t in range(OB, OE+1)), name="lambda")

# model.addConstrs((
#     mu[k, t] - lambda_k_t[k, t] >= (1/Ttol) * L[k] for k in N for t in range(OB, OE+1)
# ), name="C22")

model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    for v in model.getVars():
        if v.x > 0:
            print(f"{v.varName}: {v.x}")
else:
    print("No optimal solution found.")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-30
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i7-1365U, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2804 rows, 12600 columns and 132634 nonzeros
Model fingerprint: 0xc829c714
Variable types: 0 continuous, 12600 integer (3514 binary)
Coefficient statistics:
  Matrix range     [5e-01, 3e+02]
  Objective range  [4e-02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 3e+01]
Found heuristic solution: objective 21750.000000
Presolve removed 989 rows and 2115 columns
Presolve time: 0.11s
Presolved: 1815 rows, 10485 columns, 87635 nonzeros
Variable types: 0 continuous, 10485 integer (3513 binary)
Found heuristic solution: objective 1500.0000000

Root relaxation: objective 1.250000e+03, 23 iterations, 0.00 seconds (0.00 work

In [5]:
import pandas as pd
import numpy as np
import networkx as nx
from gurobipy import Model, GRB, quicksum
import math

# Load the Excel file
highway_node = pd.read_excel("HighwayNode.xlsx", header=None)

# Initialize R as a 3D numpy array
R = np.zeros((14, 24, 30))

# Function to clean and extract data from the CSV files
def process_csv(file_path):
    # Skip the first 3 rows and read the file from the 4th row
    data = pd.read_csv(file_path, skiprows=3, header=0, delimiter=',')
    
    # Convert all data to string and handle missing values
    data = data.astype(str).replace('nan', '')
    
    # Return the data as a NumPy array
    return data.values

# Iterate over each node
for k in range(1, 15):
    # Find rows where the 7th column equals k
    A = highway_node[highway_node.iloc[:, 6] == k].index.tolist()
    
    for i in A:
        # Get the file path from the 6th column of the selected row
        file_path = highway_node.iloc[i, 5]
        
        # Process the CSV file and extract the data
        Data = process_csv(file_path)
       
        # Iterate over each day and time period
        for n in range(30):
            for t in range(6, 24):
                try:
                    start_row = 96 * n + 4 * (t-1)
                    end_row = start_row + 4
                    R[k-1, t-1, n] += np.sum(pd.to_numeric(Data[start_row:end_row, 4]))
                except Exception as e:
                    print(f"Error processing data for node {k}, day {n}, time {t}: {e}")
                    continue

print("R_k.t.n generated successfully.")

# Define the edges (assuming node indexing starts from 1)
edges = [
    (1, 2, 1),
    (2, 3, 3.5),
    (3, 4, 14),
    (4, 5, 3.8),
    (5, 6, 17.5),
    (6, 7, 14.5),
    (7, 8, 4.1),
    (8, 9, 6.9),
    (9, 10, 15.5),
    (10, 11, 3.3),
    (11, 12, 10.7),
    (12, 13, 13),
    (13, 14, 2.7),
    (14, 1, 7.9)
]

# Create a graph
G = nx.Graph()

# Add edges to the graph
for u, v, w in edges:
    G.add_edge(u, v, weight=w)

# Ensure node labels start from 0
G = nx.convert_node_labels_to_integers(G, first_label=0)

# Initialize the distance matrix
num_nodes = G.number_of_nodes()
d = np.zeros((num_nodes, num_nodes))

# Initialize the indicator matrix
a = np.zeros((num_nodes, num_nodes, num_nodes), dtype=int)

# Compute shortest paths and distances using Dijkstra's algorithm
for i in range(num_nodes):
    # Use Dijkstra's algorithm to compute shortest paths from node i
    lengths, paths = nx.single_source_dijkstra(G, i)
    
    for j in range(num_nodes):
        if i != j:
            # Distance between node i and node j
            d[i, j] = lengths[j]
            
            # Shortest path from node i to node j
            path = paths[j]
            
            # Set indicator matrix entries
            for k in range(1, len(path) - 1):  # Exclude the source (i) and target (j) nodes
                intermediate = path[k]
                a[i, j, intermediate] = 1

Tij = d / 100

# Parameters
N = [i for i in range(1, 15)]
C1, C2, C3 = 100, 50, 100
fc = lambda c: 200 + 50 * c
gc = lambda c: 300 + 300 * c
hc = lambda c: 200 + 30 * c
theta = 10
r = 30
TC = 2
TB = 1/6
Ttol = 1/2
epsilon = 0.8
w = 0.0365
OB, OE = 6, 23
observed_days = 30

# Create a new model
model = Model("EV Infrastructure Planning")

# Decision variables
A = model.addVars(N, range(1, C1+1), vtype=GRB.BINARY, name="A")
B = model.addVars(N, range(1, C2+1), vtype=GRB.BINARY, name="B")
O = model.addVars(N, range(1, C3+1), vtype=GRB.BINARY, name="O")
L = model.addVars(N, vtype=GRB.BINARY, name="L")
H = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="H")
C = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="C")
M = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="M")
N_var = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="N")
F = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="F")
E = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="E")
D = model.addVars(N, N, range(OB, OE+1), vtype=GRB.INTEGER, name="D")
G = model.addVars(N, N, range(OB, OE+1), vtype=GRB.INTEGER, name="G")
I = model.addVars(N, vtype=GRB.INTEGER, name="I")
mu = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="mu")
lambda_k_t = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="lambda")

# Objective function
model.setObjective(
    quicksum(fc(c) * A[k, c] for k in N for c in range(1, C1+1)) +
    quicksum(gc(c) * B[k, c] for k in N for c in range(1, C2+1)) +
    quicksum(hc(c) * O[k, c] for k in N for c in range(1, C3+1)) +
    quicksum(w * d[i-1, j-1] * D[i, j, t] for i in N for j in N for t in range(OB, OE+1)) +
    quicksum(w * d[i-1, j-1] * G[i, j, t] for i in N for j in N for t in range(OB, OE+1)) +
    quicksum(theta * I[k] for k in N),
    GRB.MINIMIZE
)

# Constraints
model.addConstrs((quicksum(A[k, c] for c in range(1, C1+1)) <= 1 for k in N), name="C2")
model.addConstrs((quicksum(B[k, c] for c in range(1, C2+1)) <= 1 for k in N), name="C3")
model.addConstrs((quicksum(O[k, c] for c in range(1, C3+1)) <= 1 for k in N), name="C4")

model.addConstrs((L[k] <= quicksum(A[k, c] for c in range(1, C1+1)) + quicksum(B[k, c] for c in range(1, C2+1)) for k in N), name="C5")
model.addConstrs((L[k] >= 0.5 * (quicksum(A[k, c] for c in range(1, C1+1)) + quicksum(B[k, c] for c in range(1, C2+1))) for k in N), name="C6")

# Correct indexing of constraint C7 using matrix method
L_array = np.array([L[k] for k in N])
a_array = np.array(a)
model.addConstrs((
    r * (1 + quicksum(L_array * int(a_array[i-1, j-1, k-1]) for k in N)) >= float(d[i-1, j-1])
    for i in N for j in N
), name="C7")

model.addConstrs((F[k, OB] == I[k] for k in N), name="C8")
model.addConstrs((F[k, OE] == I[k] for k in N), name="C9")
model.addConstrs((E[k, OB] == 0 for k in N), name="C10")

model.addConstrs((
    F[k, t] == F[k, t-1] - H[k, t-1] + (M[k, t-TC] if t-TC >= OB else 0) + (N_var[k, t-TC] if t-TC >= OB else 0) -
    quicksum(D[k, j, t-1] for j in N) +
    quicksum(D[j, k, t-int(math.ceil(Tij[j-1, k-1]))] if t-int(math.ceil(Tij[j-1, k-1])) >= OB else 0 for j in N)
    for k in N for t in range(OB+1, OE+1)
), name="C11")

model.addConstrs((
    E[k, t] == E[k, t-1] + H[k, t-1] - M[k, t-1] - N_var[k, t-1] -
    quicksum(G[k, j, t-1] for j in N) +
    quicksum(G[j, k, t-int(math.ceil(Tij[j-1, k-1]))] if t-int(math.ceil(Tij[j-1, k-1])) >= OB else 0 for j in N)
    for k in N for t in range(OB+1, OE+1)
), name="C12")

model.addConstrs((M[k, t] + N_var[k, t] <= E[k, t] for k in N for t in range(OB, OE+1)), name="C13")
model.addConstrs((quicksum(M[k, l] for l in range(max(OB, t-TC+1), t+1)) <= quicksum(c * A[k, c] for c in range(1, C1+1)) for k in N for t in range(OB, OE+1)), name="C14")
model.addConstrs((quicksum(N_var[k, l] for l in range(max(OB, t-TC+1), t+1)) <= quicksum(c * O[k, c] for c in range(1, C3+1)) for k in N for t in range(OB, OE+1)), name="C15")
model.addConstrs((H[k, t] <= (1/TB) * quicksum(c * B[k, c] for c in range(1, C2+1)) for k in N for t in range(OB, OE+1)), name="C16")

model.addConstrs((quicksum(D[k, j, t] for j in N) <= F[k, t] - H[k, t] for k in N for t in range(OB, OE+1)), name="C17")
model.addConstrs((quicksum(G[k, j, t] for j in N) <= E[k, t] + H[k, t] for k in N for t in range(OB, OE+1)), name="C18")

model.addConstrs((
    quicksum(F[k, t] + E[k, t] + quicksum(M[k, l] + N_var[k, l] for l in range(max(OB, t-TC+1), t+1)) +
    quicksum(D[k, j, t-int(math.ceil(Tij[k-1, j-1]))] + G[k, j, t-int(math.ceil(Tij[k-1, j-1]))] if t-int(math.ceil(Tij[k-1, j-1])) >= OB else 0 for j in N)
    for k in N) == quicksum(I[k] for k in N)
    for t in range(OB, OE+1)
), name="C19")

model.addConstrs((C[k, t] <= (quicksum(c * A[k, c] for c in range(1, C1+1)) - quicksum(M[k, l] for l in range(max(OB, t-TC+1), t+1)) - quicksum(C[k, l] for l in range(max(OB, t-TC), t)))
                 for k in N for t in range(OB, OE+1)), name="C20")

model.addConstrs((mu[k, t] == C[k, t] + H[k, t] for k in N for t in range(OB, OE+1)), name="C21")

# model.addConstrs((quicksum(R[k-1, t-1, n-1] for n in range(1, observed_days+1)) / observed_days >= lambda_k_t[k, t] for k in N for t in range(OB, OE+1)), name="lambda")

# model.addConstrs((
#     mu[k, t] - lambda_k_t[k, t] >= (1/Ttol) * L[k] for k in N for t in range(OB, OE+1)
# ), name="C22")

model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    for v in model.getVars():
        if v.x > 0:
            print(f"{v.varName}: {v.x}")
else:
    print("No optimal solution found.")


R_k.t.n generated successfully.


GurobiError: Constraint has no bool value (are you trying "lb <= expr <= ub"?)

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from gurobipy import Model, GRB, quicksum
import math

# Load the Excel file
highway_node = pd.read_excel("HighwayNode.xlsx", header=None)

# Initialize R as a 3D numpy array
R = np.zeros((14, 24, 30))

# Function to clean and extract data from the CSV files
def process_csv(file_path):
    # Skip the first 3 rows and read the file from the 4th row
    data = pd.read_csv(file_path, skiprows=3, header=0, delimiter=',')
    
    # Convert all data to string and handle missing values
    data = data.astype(str).replace('nan', '')
    
    # Return the data as a NumPy array
    return data.values

# Iterate over each node
for k in range(1, 15):
    # Find rows where the 7th column equals k
    A = highway_node[highway_node.iloc[:, 6] == k].index.tolist()
    
    for i in A:
        # Get the file path from the 6th column of the selected row
        file_path = highway_node.iloc[i, 5]
        
        # Process the CSV file and extract the data
        Data = process_csv(file_path)
       
        # Iterate over each day and time period
        for n in range(30):
            for t in range(6, 24):
                try:
                    start_row = 96 * n + 4 * (t-1)
                    end_row = start_row + 4
                    R[k-1, t-1, n] += np.sum(pd.to_numeric(Data[start_row:end_row, 4]))
                except Exception as e:
                    print(f"Error processing data for node {k}, day {n}, time {t}: {e}")
                    continue

print("R_k.t.n generated successfully.")

# Define the edges (assuming node indexing starts from 1)
edges = [
    (1, 2, 1),
    (2, 3, 3.5),
    (3, 4, 14),
    (4, 5, 3.8),
    (5, 6, 17.5),
    (6, 7, 14.5),
    (7, 8, 4.1),
    (8, 9, 6.9),
    (9, 10, 15.5),
    (10, 11, 3.3),
    (11, 12, 10.7),
    (12, 13, 13),
    (13, 14, 2.7),
    (14, 1, 7.9)
]

# Create a graph
G = nx.Graph()

# Add edges to the graph
for u, v, w in edges:
    G.add_edge(u, v, weight=w)

# Ensure node labels start from 0
G = nx.convert_node_labels_to_integers(G, first_label=0)

# Initialize the distance matrix
num_nodes = G.number_of_nodes()
d = np.zeros((num_nodes, num_nodes))

# Compute shortest paths and distances using Dijkstra's algorithm
shortest_paths = {}
for i in range(num_nodes):
    lengths, paths = nx.single_source_dijkstra(G, i)
    for j in range(num_nodes):
        if i != j:
            d[i, j] = lengths[j]
            shortest_paths[(i, j)] = paths[j]

Tij = d / 100

# Parameters
N = [i for i in range(1, 15)]
C1, C2, C3 = 100, 50, 100
fc = lambda c: 200 + 50 * c
gc = lambda c: 300 + 300 * c
hc = lambda c: 200 + 30 * c
theta = 10
r = 30
TC = 2
TB = 1/6
Ttol = 1/2
epsilon = 0.8
w = 0.0365
OB, OE = 6, 23
observed_days = 30

# Create a new model
model = Model("EV Infrastructure Planning")

# Decision variables
A = model.addVars(N, range(1, C1+1), vtype=GRB.BINARY, name="A")
B = model.addVars(N, range(1, C2+1), vtype=GRB.BINARY, name="B")
O = model.addVars(N, range(1, C3+1), vtype=GRB.BINARY, name="O")
L = model.addVars(N, vtype=GRB.BINARY, name="L")
H = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="H")
C = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="C")
M = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="M")
N_var = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="N")
F = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="F")
E = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="E")
D = model.addVars(N, N, range(OB, OE+1), vtype=GRB.INTEGER, name="D")
G = model.addVars(N, N, range(OB, OE+1), vtype=GRB.INTEGER, name="G")
I = model.addVars(N, vtype=GRB.INTEGER, name="I")
mu = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="mu")
lambda_k_t = model.addVars(N, range(OB, OE+1), vtype=GRB.INTEGER, name="lambda")

# Objective function
model.setObjective(
    quicksum(fc(c) * A[k, c] for k in N for c in range(1, C1+1)) +
    quicksum(gc(c) * B[k, c] for k in N for c in range(1, C2+1)) +
    quicksum(hc(c) * O[k, c] for k in N for c in range(1, C3+1)) +
    quicksum(w * d[i-1, j-1] * D[i, j, t] for i in N for j in N for t in range(OB, OE+1)) +
    quicksum(w * d[i-1, j-1] * G[i, j, t] for i in N for j in N for t in range(OB, OE+1)) +
    quicksum(theta * I[k] for k in N),
    GRB.MINIMIZE
)

# Constraints
model.addConstrs((quicksum(A[k, c] for c in range(1, C1+1)) <= 1 for k in N), name="C2")
model.addConstrs((quicksum(B[k, c] for c in range(1, C2+1)) <= 1 for k in N), name="C3")
model.addConstrs((quicksum(O[k, c] for c in range(1, C3+1)) <= 1 for k in N), name="C4")

model.addConstrs((L[k] <= quicksum(A[k, c] for c in range(1, C1+1)) + quicksum(B[k, c] for c in range(1, C2+1)) for k in N), name="C5")
model.addConstrs((L[k] >= 0.5 * (quicksum(A[k, c] for c in range(1, C1+1)) + quicksum(B[k, c] for c in range(1, C2+1))) for k in N), name="C6")

# Correct indexing of constraint C7
model.addConstrs((
    d[i-1, j-1] <= r * (1 + quicksum(L[k] for k in N if k-1 in shortest_paths[(i-1, j-1)]))
    for i in N for j in N if i != j
), name="C7")

model.addConstrs((F[k, OB] == I[k] for k in N), name="C8")
model.addConstrs((F[k, OE] == I[k] for k in N), name="C9")
model.addConstrs((E[k, OB] == 0 for k in N), name="C10")

model.addConstrs((
    F[k, t] == F[k, t-1] - H[k, t-1] + (M[k, t-TC] if t-TC >= OB else 0) + (N_var[k, t-TC] if t-TC >= OB else 0) -
    quicksum(D[k, j, t-1] for j in N) +
    quicksum(D[j, k, t-int(math.ceil(Tij[j-1, k-1]))] if t-int(math.ceil(Tij[j-1, k-1])) >= OB else 0 for j in N)
    for k in N for t in range(OB+1, OE+1)
), name="C11")

model.addConstrs((
    E[k, t] == E[k, t-1] + H[k, t-1] - M[k, t-1] - N_var[k, t-1] -
    quicksum(G[k, j, t-1] for j in N) +
    quicksum(G[j, k, t-int(math.ceil(Tij[j-1, k-1]))] if t-int(math.ceil(Tij[j-1, k-1])) >= OB else 0 for j in N)
    for k in N for t in range(OB+1, OE+1)
), name="C12")

model.addConstrs((M[k, t] + N_var[k, t] <= E[k, t] for k in N for t in range(OB, OE+1)), name="C13")
model.addConstrs((quicksum(M[k, l] for l in range(max(OB, t-TC+1), t+1)) <= quicksum(c * A[k, c] for c in range(1, C1+1)) for k in N for t in range(OB, OE+1)), name="C14")
model.addConstrs((quicksum(N_var[k, l] for l in range(max(OB, t-TC+1), t+1)) <= quicksum(c * O[k, c] for c in range(1, C3+1)) for k in N for t in range(OB, OE+1)), name="C15")
model.addConstrs((H[k, t] <= (1/TB) * quicksum(c * B[k, c] for c in range(1, C2+1)) for k in N for t in range(OB, OE+1)), name="C16")

model.addConstrs((quicksum(D[k, j, t] for j in N) <= F[k, t] - H[k, t] for k in N for t in range(OB, OE+1)), name="C17")
model.addConstrs((quicksum(G[k, j, t] for j in N) <= E[k, t] + H[k, t] for k in N for t in range(OB, OE+1)), name="C18")

model.addConstrs((
    quicksum(F[k, t] + E[k, t] + quicksum(M[k, l] + N_var[k, l] for l in range(max(OB, t-TC+1), t+1)) +
    quicksum(D[k, j, t-int(math.ceil(Tij[k-1, j-1]))] + G[k, j, t-int(math.ceil(Tij[k-1, j-1]))] if t-int(math.ceil(Tij[k-1, j-1])) >= OB else 0 for j in N)
    for k in N) == quicksum(I[k] for k in N)
    for t in range(OB, OE+1)
), name="C19")

model.addConstrs((C[k, t] <= (quicksum(c * A[k, c] for c in range(1, C1+1)) - quicksum(M[k, l] for l in range(max(OB, t-TC+1), t+1)) - quicksum(C[k, l] for l in range(max(OB, t-TC), t)))
                 for k in N for t in range(OB, OE+1)), name="C20")

model.addConstrs((mu[k, t] == C[k, t] + H[k, t] for k in N for t in range(OB, OE+1)), name="C21")

# Temporarily commented out the constraints involving lambda_k_t
# model.addConstrs((quicksum(R[k-1, t-1, n-1] for n in range(1, observed_days+1)) / observed_days >= lambda_k_t[k, t] for k in N for t in range(OB, OE+1)), name="lambda")
# model.addConstrs((mu[k, t] - lambda_k_t[k, t] >= (1/Ttol) * L[k] for k in N for t in range(OB, OE+1)), name="C22")

model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    for v in model.getVars():
        if v.x > 0:
            print(f"{v.varName}: {v.x}")
else:
    print("No optimal solution found.")


R_k.t.n generated successfully.


GurobiError: Constraint has no bool value (are you trying "lb <= expr <= ub"?)

A useful link for debug: [Constraint has no bool value (are you trying "lb <= expr <= ub"?)](https://support.gurobi.com/hc/en-us/articles/360039628832-Constraint-has-no-bool-value-are-you-trying-lb-expr-ub)